In [ ]:
import dlib
dlib.__version__

'19.24.0'

In [ ]:
!pip install dlib==19.22.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.4 MB 26.6 MB/s 
  Created wheel for dlib: filename=dlib-19.22.1-cp37-cp37m-linux_x86_64.whl size=4202375 sha256=178e4a2f822330921db96fdf34980d7a39c8dd1616cb20f981a23b3904e53ac7
  Stored in directory: /root/.cache/pip/wheels/10/6f/db/ef0380a66d955f9caa81cff6027fb77c0b556568e0daace292
Successfully built dlib
  Attempting uninstall: dlib
    Found existing installation: dlib 19.24.0
    Uninstalling dlib-19.24.0:
      Successfully uninstalled dlib-19.24.0


In [ ]:
import dlib
dlib.__version__

Before running the file Upload all your data set on your goole drive in a zip format

In [ ]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Téléchargement du dataset
import gdown 
url = "https://drive.google.com/uc?id=16GsQbWQfPVRQsvuCq8MwnZ1fcw5u1bNU"

out = "dataset_deepfake.zip"
gdown.download(url, out, quiet=False)

!unzip dataset_deepfake.zip

Downloading...
From: https://drive.google.com/uc?id=16GsQbWQfPVRQsvuCq8MwnZ1fcw5u1bNU
To: /content/dataset_deepfake.zip
100%|██████████| 166M/166M [00:03<00:00, 52.9MB/s]


Archive:  dataset_deepfake.zip
   creating: deepfake_dataset/
  inflating: deepfake_dataset/alqiqhnrza.mp4  
  inflating: __MACOSX/deepfake_dataset/._alqiqhnrza.mp4  
  inflating: deepfake_dataset/agjxpidzyu.mp4  
  inflating: __MACOSX/deepfake_dataset/._agjxpidzyu.mp4  
  inflating: deepfake_dataset/akfjqoantp.mp4  
  inflating: __MACOSX/deepfake_dataset/._akfjqoantp.mp4  
  inflating: deepfake_dataset/ooafcxxfrs.mp4  
  inflating: __MACOSX/deepfake_dataset/._ooafcxxfrs.mp4  
  inflating: deepfake_dataset/aeovzefbpr.mp4  
  inflating: __MACOSX/deepfake_dataset/._aeovzefbpr.mp4  
  inflating: deepfake_dataset/kvmpmhdxly.mp4  
  inflating: __MACOSX/deepfake_dataset/._kvmpmhdxly.mp4  
  inflating: deepfake_dataset/aijlttdlrj.mp4  
  inflating: __MACOSX/deepfake_dataset/._aijlttdlrj.mp4  
  inflating: deepfake_dataset/acljesxipy.mp4  
  inflating: __MACOSX/deepfake_dataset/._acljesxipy.mp4  
  inflating: deepfake_dataset/aeudhfvcvk.mp4  
  inflating: __MACOSX/deepfake_dataset/._aeudhfvcvk

In [ ]:
#To get the average frame count 
import json
import glob
import numpy as np
import cv2
import copy
#change the path accordingly
video_files =  glob.glob('deepfake_dataset/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [299, 299, 299, 300, 299, 299, 299, 299, 299, 299, 300, 299, 299, 299, 299, 299, 300, 299, 299, 299, 299, 299, 299, 300, 299, 299, 299, 299]
Total number of videos:  28
Average frame per video: 299.14285714285717


In [ ]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path) 
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image
!pip3 install face_recognition
!mkdir '/content/drive/My Drive/Face_only_data'
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm


# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          
          faces = face_recognition.batch_face_locations(frames)
          
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
          
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 31 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=599dc59b9e1544e02aba7d97e525d2a64e38d20730d7669f1aa66166993a7f51
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
create_face_videos(video_files,'/content/drive/My Drive/Face_only_data/')

# Nouvelle section